In [25]:
import os
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO


In [14]:
%pip install BeautifulSoup4


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
SCORE_DIR = "data/scores/"

In [112]:
%pip install html5lib

  Obtaining dependency information for html5lib from https://files.pythonhosted.org/packages/6c/dd/a834df6482147d48e225a49515aabc28974ad5a4ca3215c18a882565b028/html5lib-1.1-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 791.8 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
box_scores = os.listdir(SCORE_DIR)

In [17]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [96]:
%pip install datetime

  Obtaining dependency information for datetime from https://files.pythonhosted.org/packages/ff/d5/f508192a563ab7415d1efbbe8d39cb9f0e510a1f6aaee3ca7d4ffed2a194/DateTime-5.4-py3-none-any.whl.metadata
  Obtaining dependency information for zope.interface from https://files.pythonhosted.org/packages/2c/a1/32d95d02e501899bf05dae2ddc10d0437f59cab88f2254c3e40d91a13466/zope.interface-6.2-cp312-cp312-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.3 MB/s eta 0:00:00
  Obtaining dependency information for setuptools from https://files.pythonhosted.org/packages/92/e1/1c8bb3420105e70bdf357d57dd5567202b4ef8d27f810e98bb962d950834/setuptools-69.2.0-py3-none-any.whl.metadata
  Using cached setuptools-69.2.0-py3-none-any.whl.metadata (6.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 8.8 MB/s eta 0:00:00
Using cached setuptools-69.2.0-py3-none-any.whl (821 kB)

[no

In [18]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
    
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [27]:
base_cols = None
box_score = box_scores[0]

# for box_score in boc_scores:
soup = parse_html(box_score)
line_score = read_line_score(soup)
teams = list(line_score["team"])

summaries = []
for team in teams:
    basic = read_stats(soup, team, "basic")
    advanced = read_stats(soup, team, "advanced")

    totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
    totals.index = totals.index.str.lower()

    maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
    maxes.index = maxes.index.str.lower() + "_max"

    summary = pd.concat([totals,maxes])

    if base_cols is None:
        base_cols = list(summary.index.drop_duplicates(keep = "first"))
        base_cols = [b for b in base_cols if "bpm" not in b]

    summary = summary[base_cols]

    summaries.append(summary)

summary = pd.concat(summaries, axis =1).T

game = pd.concat([summary, line_score], axis = 1)

game["home"] = [0,1]
game_opp = game.iloc[::-1].reset_index()
game_opp.columns += "_opp"

full_game = pd.concat([game, game_opp], axis = 1)



/var/folders/8d/ycl0fys12wq90tgn83r138pw0000gn/T/ipykernel_57030/3703257073.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]


In [26]:
def read_stats(soup, team, stat):
    df = pd.read_html(StringIO(str(soup)), attrs={"id" : f"box-{team}-game-{stat}"}, index_col = 0)[0]
    df = df.apply(pd.to_numeric, errors = "coerce")
    return df

In [28]:
def read_line_score(soup):
    line_score = pd.read_html(StringIO(str(soup)), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols

    line_score = line_score[["team", "total"]]
    return line_score


In [30]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [31]:
full_game

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp
0,240.0,240.0,38.0,72.0,0.528,16.0,37.0,0.432,18.0,21.0,...,33.4,9.2,4.1,26.2,26.8,155.0,123.0,MIA,107,1
1,240.0,240.0,40.0,84.0,0.476,14.0,39.0,0.359,13.0,15.0,...,30.6,3.9,8.1,41.0,37.3,160.0,121.0,POR,110,0


In [33]:
base_cols = None
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals,maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep = "first"))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols]

        summaries.append(summary)

    summary = pd.concat(summaries, axis =1).T

    game = pd.concat([summary, line_score], axis = 1)

    game["home"] = [0,1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis = 1)

    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)

    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")



100 / 10198
200 / 10198
300 / 10198
400 / 10198
500 / 10198
600 / 10198
700 / 10198
800 / 10198
900 / 10198
1000 / 10198
1100 / 10198
1200 / 10198
1300 / 10198
1400 / 10198
1500 / 10198
1600 / 10198
1700 / 10198
1800 / 10198
1900 / 10198
2000 / 10198
2100 / 10198
2200 / 10198
2300 / 10198
2400 / 10198
2500 / 10198
2600 / 10198
2700 / 10198
2800 / 10198
2900 / 10198
3000 / 10198
3100 / 10198
3200 / 10198
3300 / 10198
3400 / 10198
3500 / 10198
3600 / 10198
3700 / 10198
3800 / 10198
3900 / 10198
4000 / 10198
4100 / 10198
4200 / 10198
4300 / 10198
4400 / 10198
4500 / 10198
4600 / 10198
4700 / 10198
4800 / 10198
4900 / 10198
5000 / 10198
5100 / 10198
5200 / 10198
5300 / 10198
5400 / 10198
5500 / 10198
5600 / 10198
5700 / 10198
5800 / 10198
5900 / 10198
6000 / 10198
6100 / 10198
6200 / 10198
6300 / 10198
6400 / 10198
6500 / 10198
6600 / 10198
6700 / 10198
6800 / 10198
6900 / 10198
7000 / 10198
7100 / 10198
7200 / 10198
7300 / 10198
7400 / 10198
7500 / 10198
7600 / 10198
7700 / 10198
7800 / 1

ValueError: No tables found

In [34]:
games_df = pd.concat(games,ignore_index=True)

In [35]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,38.0,72.0,0.528,16.0,37.0,0.432,18.0,21.0,...,26.2,26.8,155.0,123.0,MIA,107,1,2023,2022-11-07,True
1,240.0,240.0,40.0,84.0,0.476,14.0,39.0,0.359,13.0,15.0,...,41.0,37.3,160.0,121.0,POR,110,0,2023,2022-11-07,False
2,240.0,240.0,41.0,78.0,0.526,8.0,24.0,0.333,15.0,19.0,...,28.6,41.1,250.0,125.0,DAL,90,1,2023,2022-12-14,True
3,240.0,240.0,29.0,74.0,0.392,13.0,38.0,0.342,19.0,26.0,...,12.6,33.0,183.0,110.0,CLE,105,0,2023,2022-12-14,False
4,240.0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20007,240.0,240.0,33.0,86.0,0.384,7.0,23.0,0.304,18.0,25.0,...,29.6,30.2,152.0,106.0,TOR,102,0,2016,2015-11-03,False
20008,240.0,240.0,37.0,100.0,0.370,12.0,40.0,0.300,3.0,5.0,...,18.3,55.0,169.0,101.0,MIL,117,1,2020,2019-12-22,False
20009,240.0,240.0,44.0,104.0,0.423,15.0,44.0,0.341,14.0,17.0,...,25.0,33.5,200.0,126.0,IND,89,0,2020,2019-12-22,True
20010,240.0,240.0,35.0,79.0,0.443,15.0,27.0,0.556,10.0,12.0,...,50.0,32.2,122.0,110.0,ORL,91,1,2019,2019-01-25,True


In [36]:
games_df.to_csv("nba_games.csv")